In [1]:
import pandas as pd
import statsmodels.formula.api as smf
import numpy as np

data = pd.read_csv("/workspaces/bakery_sales_prediction/0_DataPreparation/00_data/data_long.csv")
#print(data.head())
#data brot:
df= data[data["Warengruppe_4"] == 1].copy()
data_wetter = pd.read_csv("/workspaces/bakery_sales_prediction/2_BaselineModel/02_Franz/weather_data_with_flags.csv")


df = df.drop(columns=["Warengruppe_1", "Warengruppe_2", "Warengruppe_3", "Warengruppe_4", "Warengruppe_5", "Warengruppe_6"])
df = df.drop(columns=["Datum"])
print(df)



          Umsatz  Temp_Very_Cold  Temp_Cold  Temp_Mild  Temp_Warm  Temp_Hot  \
5457   65.890169               0          0          1          0         0   
5458   74.543917               0          0          1          0         0   
5459   69.262728               0          0          0          1         0   
5460   61.490175               0          0          1          0         0   
5461   86.759861               0          0          1          0         0   
...          ...             ...        ...        ...        ...       ...   
7218   65.563734               0          0          0          0         1   
7219   54.047422               0          0          0          0         1   
7220  128.952852               0          0          0          1         0   
7221   71.822977               0          0          0          0         1   
7222   57.102795               0          0          0          0         1   

      Cloud_Clear  Cloud_Partly_Cloudy  Cloud_Cloud

In [2]:
# 1. Schritt 1: Untersuche lineare Korrelation zwischen den einzelnen Features mit der Zielvariable
# mit Korrelationsmatrix
import pandas as pd

# Korrelationsmatrix erstellen
correlation_matrix = df.corr()

# Korrelation mit der Zielvariable
correlation_with_target = correlation_matrix["Umsatz"].sort_values(ascending=False)
print(correlation_with_target)




Umsatz                    1.000000
Sonntag                   0.534262
Monat_2                   0.302366
Feiertag                  0.161383
Winter                    0.142874
Temp_Cold                 0.087436
Monat_8                   0.055671
wetter_sehr_schlecht      0.034816
Markt                     0.032351
Monat_10                  0.030682
Cloud_ok                  0.028084
Heimspiel                 0.026833
Cloud_Partly_Cloudy       0.018067
Weather_Light_Issues      0.017940
Cloud_Clear               0.014779
Monat_9                   0.005099
Wind_Moderate             0.004766
Monat_3                   0.002680
Wind_ok                   0.001919
Wind_Strong              -0.001919
Herbst                   -0.003001
Wind_Light               -0.003701
Temp_Mild                -0.004113
zwischen_den_jahren      -0.006089
Weather_Moderate         -0.007113
Temp_Very_Cold           -0.007295
Number_of_ships_scaled   -0.008914
Number_of_ships          -0.008914
Weather_Good        

In [3]:
from sklearn.feature_selection import mutual_info_regression

# Nicht-lineare Zusammenhänge analysieren
X = df.drop("Umsatz", axis=1)
y = df["Umsatz"]
mi = mutual_info_regression(X, y)
mi_series = pd.Series(mi, index=X.columns).sort_values(ascending=False)
print(mi_series)


Sonntag                   0.146606
Werktag                   0.090289
VPI                       0.060827
Monat_2                   0.033423
Winter                    0.015709
Mittwoch                  0.015191
Number_of_ships           0.014301
Sommer                    0.014047
Number_of_ships_scaled    0.013960
Feiertag                  0.013689
Donnerstag                0.013484
Monat_6                   0.009207
Monat_4                   0.009091
Weather_Moderate          0.008915
Ship                      0.008891
Monat_1                   0.008469
Monat_10                  0.007927
Montag                    0.007406
is_holiday                0.007143
wetter_sehr_schlecht      0.006979
Wind_Strong               0.006097
Wind_ok                   0.006097
Weather_Good              0.006020
Herbst                    0.005819
KielerWoche               0.005209
Monat_7                   0.005127
wetter_sehr_schön         0.004855
Heimspiel                 0.004417
Dienstag            

In [4]:
# Features mit geringer linearer und nichtlinearer Korrelation finden
low_linear_corr_features = correlation_with_target[correlation_with_target.abs() < 0.1].index
low_nonlinear_corr_features = mi_series[mi_series < 0.02].index

# Schnittmenge der Features mit beiden Bedingungen
low_corr_features = set(low_linear_corr_features).intersection(low_nonlinear_corr_features)

# Ausgabe der Features
#print("Features mit geringer linearer und nichtlinearer Korrelation:", list(low_corr_features))

# DataFrame erstellen für die Ausgabe in Tabellenform
low_corr_features_table = pd.DataFrame({
    "Feature": list(low_corr_features),
    "Lineare Korrelation": [correlation_with_target[feature] for feature in low_corr_features],
    "Nichtlineare Korrelation": [mi_series[feature] for feature in low_corr_features]
})


# Tabelle nach der linearen Korrelation sortieren
low_corr_features_table = low_corr_features_table.sort_values(by="Lineare Korrelation", ascending = False)

# Ausgabe der sortierten Tabelle
print(low_corr_features_table)



                   Feature  Lineare Korrelation  Nichtlineare Korrelation
5                Temp_Cold             0.087436                  0.000000
29                 Monat_8             0.055671                  0.002099
8     wetter_sehr_schlecht             0.034816                  0.006979
6                    Markt             0.032351                  0.000000
11                Monat_10             0.030682                  0.007927
24                Cloud_ok             0.028084                  0.000000
18               Heimspiel             0.026833                  0.004417
0      Cloud_Partly_Cloudy             0.018067                  0.000000
27    Weather_Light_Issues             0.017940                  0.003229
15             Cloud_Clear             0.014779                  0.000000
2                  Monat_9             0.005099                  0.000000
31           Wind_Moderate             0.004766                  0.000000
44                 Monat_3            

In [5]:

import numpy as np

# Korrelationsmatrix berechnen
corr_matrix = df.corr()

# Stark korrelierte Features (> 0.5) finden
high_corr = corr_matrix.abs() > 0.7
np.fill_diagonal(high_corr.values, False)  # Hauptdiagonale ignorieren
correlated_features = [column for column in high_corr.columns if any(high_corr[column])]
print(f"Stark korrelierte Features:", correlated_features)

Stark korrelierte Features: ['Cloud_Cloudy', 'Wind_Light', 'Wind_Moderate', 'Wind_Strong', 'Number_of_ships', 'Ship', 'Weihnachtsmarkt', 'Monat_12', 'Number_of_ships_scaled', 'Cloud_ok', 'Wind_ok']


In [6]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# VIF für alle Features berechnen
X = df.drop("Umsatz", axis=1)
vif = pd.DataFrame()
vif["Feature"] = X.columns
vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif)



/workspaces/bakery_sales_prediction/.venv/lib/python3.12/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


                   Feature           VIF
0           Temp_Very_Cold           inf
1                Temp_Cold           inf
2                Temp_Mild  1.425403e+06
3                Temp_Warm  3.665432e+04
4                 Temp_Hot           inf
5              Cloud_Clear  2.057949e+05
6      Cloud_Partly_Cloudy  1.918159e+06
7             Cloud_Cloudy  5.406396e+04
8               Wind_Light  2.421758e+04
9            Wind_Moderate  7.024139e+04
10             Wind_Strong  4.263569e+06
11            Weather_Good  1.189357e+06
12    Weather_Light_Issues  8.639560e+03
13        Weather_Moderate  3.303501e+05
14          Weather_Severe  1.484907e+06
15             KielerWoche  1.447167e+00
16                  Montag  4.469525e+06
17                Dienstag  2.180411e+06
18                Mittwoch  1.571563e+04
19              Donnerstag  1.094983e+05
20                 Freitag  3.100179e+05
21                 Samstag  5.895458e+05
22                 Sonntag  6.902084e+05
23              

In [7]:
# suche miteinander korrelierrende features


In [8]:

# Korrelationsmatrix berechnen
corr_matrix = df.corr()

# Schwellenwert für "stark korreliert"
threshold = 0.4

# Finde die Paare mit starker Korrelation
strong_corr_pairs = set()

# Iteriere über die obere Dreiecksmatrix der Korrelationsmatrix
for i in range(len(corr_matrix.columns)):
    for j in range(i):
        if abs(corr_matrix.iloc[i, j]) > threshold:  # Korrelation über dem Schwellenwert
            strong_corr_pairs.add((corr_matrix.columns[i], corr_matrix.columns[j]))

# Anzeigen der Paare starker Korrelationen
print("Stark korrelierte Feature-Paare:")
print(strong_corr_pairs)


Stark korrelierte Feature-Paare:
{('Monat_4', 'Markt'), ('Winter', 'Temp_Cold'), ('Sonntag', 'Umsatz'), ('Monat_5', 'Frühling'), ('Wind_ok', 'Wind_Strong'), ('Wind_ok', 'Wind_Moderate'), ('Monat_11', 'Herbst'), ('Monat_12', 'Winter'), ('Winter', 'Temp_Mild'), ('Werktag', 'Umsatz'), ('Monat_12', 'Weihnachtsmarkt'), ('Cloud_Cloudy', 'Cloud_Partly_Cloudy'), ('Cloud_Cloudy', 'Cloud_Clear'), ('wetter_sehr_schön', 'Temp_Hot'), ('Number_of_ships_scaled', 'Number_of_ships'), ('Wind_Moderate', 'Wind_Light'), ('Cloud_ok', 'Cloud_Cloudy'), ('Monat_6', 'KielerWoche'), ('Sommer', 'Temp_Warm'), ('wetter_sehr_schlecht', 'Temp_Very_Cold'), ('Monat_8', 'Sommer'), ('Werktag', 'Samstag'), ('Weather_Light_Issues', 'Weather_Good'), ('Monat_10', 'Herbst'), ('Sommer', 'Temp_Cold'), ('Monat_1', 'Winter'), ('Monat_9', 'Herbst'), ('Monat_7', 'Sommer'), ('Number_of_ships_scaled', 'Ship'), ('Monat_3', 'Frühling'), ('Monat_4', 'Frühling'), ('Temp_Mild', 'Temp_Cold'), ('Cloud_ok', 'Cloud_Partly_Cloudy'), ('Cloud_ok